In [18]:
from zipfile import ZipFile
from matplotlib import pyplot as plt 
from PIL import Image
import cv2
import numpy as np

In [2]:
source_file = './dataset/brain-cancer.zip'

In [3]:
zip = ZipFile(source_file)

In [31]:
def get_label(filename):
    parts = filename.lower().split('/')
    result = None
    if 'yes' in parts:
        result = 1
    elif 'no' in parts:
        result = 0
    else:
        pass
    
    return result

assert get_label('brain_tumor_dataset/no/30 no.jpg') == 0
assert get_label('brain_tumor_dataset/NO/31 no.jpg') == 0
assert get_label('yes/Y95.jpg') == 1
assert get_label('brain_tumor_dataset/yes/Y155.JPG') == 1
assert get_label('brain_tumor_dataset/Y155.JPG') == None

In [35]:
# process
data_results= []
for filename in zip.namelist():
    label = get_label(filename)
    img_data = Image.open(zip.open(filename))
    img_array = np.array(img_data, np.int8)
    data_results.append(
        {'filename': filename, 
         'label': label, 
         'img_array': img_array}
    )